<a href="https://colab.research.google.com/github/mcnica89/DNNs/blob/main/Copy_of_Example_1_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter
!pip install ortools

     |████████████████████████████████| 153kB 28.3MB/s 
     |████████████████████████████████| 14.4MB 199kB/s 
     |████████████████████████████████| 1.0MB 31.2MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
import jax.numpy as jnp
import numpy as np
from jax import random
import jax as jx
from jax.ops import index, index_update


import pandas as pd
import scipy as sp
import jax.scipy as jsp
import math
import matplotlib as mpl
import statsmodels.api as sm
from sklearn import linear_model
import time
import xlsxwriter

from jax.scipy import optimize
from ortools.linear_solver import pywraplp

from sklearn.linear_model import LinearRegression
from scipy.optimize import Bounds

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**PROCESS:**
0. generate n starting points


*BEGIN ALGORITHM LOOP*
1. generate m walkers for each point \**In Parallel*
>for each q, a member of xPoints or yPoints , make vector of steps within stepsize, and add q to it to create vector of walkers
>> should have 2*n vectors of m walkers

2. calculate fitness for each vector of walkers \**In Parallel with other vectors*

> For each member of each vector, given corresponding vector: \**In Parallel with other members*
> > 2.1 optimize P1 given P2 using looping bounded optimizer

> > 2.2 optimize P2 given P1 using looping bounded optimizer 

> > 2.3 calculate distance to shadow point (pick a norm)

3. for each vector of walkers, find the weighted average and replace corresponding point from xPoints


**CONSTRUCTION**

xPoints, yPoints - vectors of size n, containing the coords of the points in this generation

xWalkers, yWalkers - vectors of size n \* m containing the coordinates of all the walkers in this generation associated with each point

optimizeX(y) and optimizeY(x) - functions that intake 'other' coord and return the optimum for the given coord under their objective function and state of 'other'
>These require function to find the bounds, and a function to optimize within those bounds

jaxPenalty(x,y) - function should intake a point, utilize optimizeX and optimizeY to find shadow point, then return the norm to that point. 
> Should be vMapped to intake xPoints and yPoints or xWalkers and yWalkers and return a vector of fitness values

# 0. Initialize

In [ ]:
cols = 20
numSelect = 10

def make_key():
  seed = np.random.randint(2**16 - 1)
  return random.PRNGKey(seed)

key = make_key()
key, subkey = random.split(key)

xPoints = random.uniform(key, shape=(3,))
yPoints = random.uniform(subkey, shape=(3,))
print(xPoints)
print(yPoints)


[0.05582583 0.89135015 0.7251289 ]
[0.3155619  0.43022215 0.80983317]


In [ ]:
yBigger = (xPoints < yPoints)*1
ySmaller = (yBigger-1)*(-1)
ySmaller

DeviceArray([0, 1, 0], dtype=int32)

#Prep Optimizer

In [ ]:
def goldenSearch(lowB, upB, f, maxSpace):
  R = (5**0.5 - 1) / 2
  D = R * (upB - lowB)
  x1 = lowB + D
  x2 = upB - D
  f1 = f(x1)
  f2 = f(x2)



  for i in range(5):
    f1Lower = (f1 < f2)*1
    if f1 < f2:
      lowB = x2
      x2 = x1
      f2 = f1
      x1 = lowB + R*(upB - lowB)
      f1 = f(x1)
    else:
      upB = x1
      x1 = x2
      f1 = f2
      x2 = upB - R*(upB - lowB)
      f2 = f(x2)

    if f1 < f2:
      xopt = x1

    else:
      xopt = x2


  
  return xopt

In [ ]:
def goldenVecSearch(lowB, upB, f):
  R = (5**0.5 - 1) / 2
  D = R * (upB - lowB)
  x1 = lowB + D
  x2 = upB - D
  f1 = f(x1)
  f2 = f(x2)
  
  iter = 0
  #for i  in range(50):
  while max(abs(f1-f2)) > 0.0001:
    f1Lower = (f1 < f2)*1
    f2Lower = (f1Lower - 1)*(-1)
    
    lowB = x2*f1Lower + lowB*f2Lower
    x2 = x1* f1Lower + x2*f2Lower
    f2 = f1*f1Lower + f2*f2Lower
    x1 = (lowB + R*(upB-lowB))*f1Lower + x1*f2Lower
    f1 = f(x1)*f1Lower + f1*f2Lower

    upB = x1*f2Lower + upB*f1Lower
    x1 = x2*f2Lower + x1*f1Lower
    f1 = f2*f2Lower + f1*f1Lower
    x2 = (upB - R*(upB-lowB))*f2Lower + x2*f1Lower
    f2 = f(x2)*f2Lower + f2*f1Lower

    iter = iter+1

  f1Lower = (f1 < f2)*1
  f2Lower = (f1Lower - 1)*(-1)
  xOpt = x1*f1Lower + x2*f2Lower
  print(iter)

  return xOpt

In [ ]:
lowB = np.array([0, 0, 0, 0])
upB = np.array([1,0.5,0.7,0.2])
def func(x):
  return (0.5-x)**2

pleaseWork = goldenVecSearch(lowB, upB, func)
pleaseWork

12


array([0.49963343, 0.49940688, 0.50009062, 0.19976275])

## New optimizers


In [ ]:
xMinGConstr = 0
xMaxGConstr = 1

yMinGConstr = 0
yMaxGConstr = 1

def findMaxX(y):
    maxX = 1-y 
   # if maxX <=0 : maxX = 0.001
    return maxX

def findMaxY(x):
    maxY = 1-x 
   # if maxY <= 0: maxY = 0.001
    return maxY


def optimizeX(y): #Inputs: the other variable value to create bounds, objective function to optimize
    
    #Constrain my variable by other
    myLim = findMaxX(y)
    myBounds = (xMinGConstr, myLim)

    #ReDefine x function based on y
    def funcx(x):
        return (x-1)**2
  
    #Solve x based on xConstraint
    
    #NEW SOLVER
    myOpt = goldenSearch(xMinGConstr, myLim, funcx, 0.001)

    return myOpt


def optimizeY(x,z): #Inputs: the other variable value to create bounds, objective function to optimize
    
    #Constrain my variable by other
    myLim = findMaxY(x)
    myBounds = (yMinGConstr, myLim)

    #ReDefine x function based on y
    def funcy(y):
        return (y-0.5)**2
    
    #Solve x based on xConstraint
    
    #NEW SOLVER
    myOpt = goldenSearch(yMinGConstr, myLim, funcy, 0.001)

    return myOpt

## Old Optimizers


In [ ]:
xMinGConstr = 0
xMaxGConstr = 1

yMinGConstr = 0
yMaxGConstr = 1

def oldFindMaxX(y):
    maxX = max(0.0001, 1-y) 
    return maxX

def oldFindMaxY(x):
    maxY = max(0.0001, 1-x)
    return maxY


def oldOptimizeX(y): #Inputs: the other variable value to create bounds, objective function to optimize
    
    #Constrain my variable by other
    myLim = findMaxX(y)
    myBounds = (xMinGConstr, myLim)

    #ReDefine x function based on y
    def funcx(x):
        return (x-1)**2
  
    #Solve x based on xConstraint

    #OLD SOLVER BELOW
    mySolver = sp.optimize.minimize_scalar(funcx, bounds=myBounds, method='bounded', tol=None, options=None)
    myOpt = mySolver.x

    return myOpt


def oldOptimizeY(x): #Inputs: the other variable value to create bounds, objective function to optimize
    
    #Constrain my variable by other
    myLim = findMaxY(x)
    myBounds = (yMinGConstr, myLim)

    #ReDefine x function based on y
    def funcy(y):
        return (y-0.5)**2
    
    #Solve x based on xConstraint
    
    #OLD SOLVER BELOW
    mySolver = sp.optimize.minimize_scalar(funcy, bounds=myBounds, method='bounded', tol=None, options=None)
    myOpt = mySolver.x

    return myOpt

In [ ]:
#this is the original penalty function
def penalty(x,y):

  xOpt = oldOptimizeX(y)
  yOpt = oldOptimizeY(x)

  xDist = abs(x - xOpt)
  yDist = abs(y - yOpt)

  return math.sqrt(xDist**2 + yDist**2)

def findFitness(xVec, yVec):
  vec = np.ones(size)
  for i in range(size):
    vec[i] = penalty(xVec[i], yVec[i])
  return vec

Ideally, we vectorize this penalty so that we can find the shadow points for all points at once. 


In [ ]:
#this is the vectorized penalty function
from jax import vmap
def jaxPenalty(x: float, y: float) -> float:
  xOpt = optimizeX(y)
  yOpt = optimizeY(x)

  xDist = abs(x - xOpt)
  yDist = abs(y - yOpt)

  return math.sqrt(xDist**2 + yDist**2)

mapped_penalty=vmap(jaxPenalty)

penaltyVec = mapped_penalty(xPoints,yPoints)

ConcretizationTypeError: ignored

In [ ]:
testVec = np.ones(4)
testVec = jx.ops.index_update(testVec, jx.ops.index [0], 4)
testVec

DeviceArray([4., 1., 1., 1.], dtype=float32)

In [ ]:
def funcx(x):
    return (x-1)**2
x = goldenSearch(0,1, funcx, 0.001)
x

0.9988137587103578

In [ ]:
def walkerGenerate(pointVec, size:int, genCount:int, stepSize:float) -> float: #point = point for which you want walkers. size = number of walkers to generate. genCount = current generation number. stepSize = max distance from point each walker can be
  key = make_key()
  trueStepSize = stepSize*(0.99**genCount)
  steps = random.uniform(key, shape=(size,10))*trueStepSize*2
  steps = steps - trueStepSize
  return steps+pointVec

walkerVec1 = walkerGenerate(2.5, 20, 6, 0.1)
walkerVec1
print(np.shape(walkerVec1))

(20, 10)


In [ ]:


inter = 1
intmax = 1000

size = 10
walkerCount = 5
stepSize = 0.1

t0 = time.time()

key = make_key()
key, subkey = random.split(key)

#Generate initial points
xPoints = random.uniform(key, shape=(size,))
yPoints = random.uniform(subkey, shape=(size,))


while inter < intmax:
  inter += 1
  #1. Generate walkers

  #***NEED TO MODIFY TO MAKE PARALLEL***
  for i in range(size):
    xWalkerVec[i] = walkerGenerate(xPoints[i], walkerCount, inter, stepSize)
    yWalkerVec[i] = walkerGenerate(yPoints[i], walkerCount, inter, stepSize)    

  #2. Find Fitness of walkers
    #***NEED TO MODIFY TO MAKE PARALLEL, once jaxPenalty works***
    fitVec[i] = findFitness(xWalkerVec[i], yWalkerVec[i])

  #3. Update the points with weighted average of walkers


TypeError: ignored

In [ ]:
inter = 1
intmax = 1000

numPoints = 10
size = 5
walkerCount = 5
stepSize = 0.1

t0 = time.time()

key = make_key()
key, subkey = random.split(key)

#Generate initial points
xPoints = random.uniform(key, shape=(numPoints,))
yPoints = random.uniform(subkey, shape=(numPoints,))

xWalkerVec = np.ones((10,5))
yWalkerVec = np.ones((10,5))

fitVec = np.ones((10,5))

for i in range(size):
  xWalkerVec[i,:] = walkerGenerate(xPoints[i], walkerCount, inter, stepSize)
  yWalkerVec[i,:] = walkerGenerate(yPoints[i], walkerCount, inter, stepSize)   

  fitVec[i,:] = findFitness(xWalkerVec[i], yWalkerVec[i])

IndexError: ignored

In [ ]:
fitVec

array([[0.14586225, 0.2170533 , 0.26602379, 0.27603427, 0.40338524],
       [0.93167645, 0.83803068, 0.86090947, 0.68473887, 0.77651289],
       [0.7535081 , 0.77203891, 0.81803076, 0.89743169, 1.01621576],
       [0.37692605, 0.25673337, 0.41238675, 0.48714684, 0.39155136],
       [0.22791252, 0.47991437, 0.41961006, 0.39737105, 0.18015028],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ]])